# 📊 Prévision de la production hydro-électrique

Dans cette section, nous allons nous concentrer sur **la centrale hydro-électrique située dans la région de Montpellier**.  
L’objectif est de prédire la **production quotidienne d’énergie** à partir de données environnementales et hydrologiques locales.

- `date_obs_elab` : date de l’observation (YYYY-MM-DD)  
- `prod_hydro` : production d’énergie (kW)  


## Import 

In [8]:
import pandas as pd
import matplotlib.pyplot as plt

## 1. Chargement des données

In [9]:
df = pd.read_csv("~/production_enr/data/prod_hydro.csv")
display(df.head(6))

,date_obs_elab,prod_hydro
0,2022-08-01,200.0
1,2022-09-01,200.0
2,2022-10-01,200.0
3,2022-11-01,200.0
4,2022-12-01,0.0
5,2023-01-01,200.0


## 2. Dimensions et types de variables

In [10]:
df.dtypes

date_obs_elab     object
prod_hydro       float64
dtype: object

## 3. Valeurs manquantes

In [11]:
missing = df.isna().sum()

print(f"Missing values: {missing}")

Missing values: date_obs_elab    0
prod_hydro       0
dtype: int64


## 4. Statistiques descriptives

In [12]:
display(df.describe())

,prod_hydro
count,996.000000
mean,192.138907
std,71.216446
min,0.000000
25%,200.000000
50%,200.000000
75%,200.000000
max,2000.000000


## 5. Valeurs aberrantes
- **d’une centrale hydro (200 kW nominaux)**

In [13]:
df_clean = df[df['prod_hydro'] <= 200]
display(df_clean)
# j'ai supprimé la valeur abberante

,date_obs_elab,prod_hydro
0,2022-08-01,200.0
1,2022-09-01,200.0
2,2022-10-01,200.0
3,2022-11-01,200.0
4,2022-12-01,0.0
...,...,...
991,2025-02-19,200.0
992,2025-02-20,200.0
993,2025-02-21,200.0
994,2025-02-22,0.0
